In [12]:

lakehouse_silver_table=spark.conf.get('spark.executorEnv.lakehouse_silverTable')

print(lakehouse_silver_table)

StatementMeta(, eb8c1da8-16b0-4e5d-8a87-8631dfa2a621, 14, Finished, Available, Finished)

abfss://Fabric_E2E@onelake.dfs.fabric.microsoft.com/Lakehouse_Silver_.Lakehouse/Tables


In [3]:
patients_df = spark.read.format("csv").option("header","true").load("Files/raw/patients.csv")
# df now is a Spark DataFrame containing CSV data from "Files/raw/patients.csv".
display(patients_df)
patients_df.count()

StatementMeta(, eb8c1da8-16b0-4e5d-8a87-8631dfa2a621, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 99d1f9b6-1249-40b0-87a4-a870c4f9fe09)

974

In [4]:
patients_transformed=patients_df.dropna(subset=['id'])
display(patients_transformed)
patients_transformed.count()


StatementMeta(, eb8c1da8-16b0-4e5d-8a87-8631dfa2a621, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0dabbf2a-9732-43e6-aa15-bca48317dce1)

974

In [5]:
patients_cleaned=patients_transformed.dropDuplicates()

StatementMeta(, eb8c1da8-16b0-4e5d-8a87-8631dfa2a621, 7, Finished, Available, Finished)

In [6]:
patients_clean=patients_cleaned.withColumnRenamed('id','DIM_patientId')

StatementMeta(, eb8c1da8-16b0-4e5d-8a87-8631dfa2a621, 8, Finished, Available, Finished)

In [7]:
patients_clean.printSchema()

StatementMeta(, eb8c1da8-16b0-4e5d-8a87-8631dfa2a621, 9, Finished, Available, Finished)

root
 |-- DIM_patientId: string (nullable = true)
 |-- BIRTHDATE: string (nullable = true)
 |-- DEATHDATE: string (nullable = true)
 |-- PREFIX: string (nullable = true)
 |-- FIRST: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- SUFFIX: string (nullable = true)
 |-- MAIDEN: string (nullable = true)
 |-- MARITAL: string (nullable = true)
 |-- RACE: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- BIRTHPLACE: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- ZIP: string (nullable = true)
 |-- LAT: string (nullable = true)
 |-- LON: string (nullable = true)



In [8]:
from pyspark.sql.functions import to_date, col

patients_final = patients_clean.withColumn("BIRTHDATE", to_date(col("BIRTHDATE"), "yyyy-MM-dd")) \
                        .withColumn("DEATHDATE", to_date(col("DEATHDATE"), "yyyy-MM-dd"))


StatementMeta(, eb8c1da8-16b0-4e5d-8a87-8631dfa2a621, 10, Finished, Available, Finished)

In [10]:
patients_final.printSchema()

StatementMeta(, eb8c1da8-16b0-4e5d-8a87-8631dfa2a621, 12, Finished, Available, Finished)

root
 |-- DIM_patientId: string (nullable = true)
 |-- BIRTHDATE: date (nullable = true)
 |-- DEATHDATE: date (nullable = true)
 |-- PREFIX: string (nullable = true)
 |-- FIRST: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- SUFFIX: string (nullable = true)
 |-- MAIDEN: string (nullable = true)
 |-- MARITAL: string (nullable = true)
 |-- RACE: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- BIRTHPLACE: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- ZIP: string (nullable = true)
 |-- LAT: string (nullable = true)
 |-- LON: string (nullable = true)



In [13]:
patients_final.write.mode("overwrite").format("delta").save(f"{lakehouse_silver_table}/DIM_Patients")

StatementMeta(, eb8c1da8-16b0-4e5d-8a87-8631dfa2a621, 15, Finished, Available, Finished)